This notebook enters data in mode shares data (trip length)

Process & philosphy:
1. Data in Kenya for mode shares is very limited. We have the GIZ Trigger spread sheet, and the overall road pkm value from UNDESA.
2. We use the vehicle-km in GIZ Trigger, multiplied by the assumed occupancy rates of vehicles (Table 7) in the GIZ/UoN report, to generate pkm values for each mode. These do not match the UNDESA values, so they are scaled to match the UNDESA total.
3. We then take the UK data and sum td per person for each mode. We divide the total pkm by mode in Kenya by that number, to generate a scaling dictionary for all modes.
4. The distribution of trips by distance for each mode is assumed the same for the UK and Kenya, in the absence of other data.

In [21]:
import pandas as pd

In [22]:
mode_shares = pd.read_excel(f'./../../TEAM-Kenya_data_out/original-team-uk-data/Mode_Shares_TripLength.xlsx')
mode_shares

,ScenarioID,CountryID,PolicyID,Year,triplenID,modeID,td
0,1,9,0,2012,1,1,56.221958
1,1,9,0,2012,1,2,1.273087
2,1,9,0,2012,1,3,12.423084
3,1,9,0,2012,1,4,7.458504
4,1,9,0,2012,1,5,0.038439
...,...,...,...,...,...,...,...
8539,1,9,0,2100,8,8,221.572479
8540,1,9,0,2100,8,9,60.330936
8541,1,9,0,2100,8,10,3.761879
8542,1,9,0,2100,8,11,14.086174


In [23]:
# create blank mode_scaling dictionary
mode_scaling = {}
for d in range(1,13):
    mode_scaling[d] = 0 

# create modeID lookup table from table to TEAM modes.
modeID_lookup = {1: 'Walk', 
                 2: 'Bike', 
                 3: 'CarDriver', 
                 4: 'CarPass', 
                 5: 'Moto', 
                 6: 'LocalBus', 
                 7: 'Coach', 
                 8: 'RailTube',
                 9: 'OtherPriv', 
                 10: 'Taxi',
                 11: 'Air', 
                 12: 'OtherPub'}

In [24]:
# use data from GIZ Trigger (2021) --> vkm by mode, with occupancy factors from https://www.changing-transport.org/wp-content/uploads/2018_GIZ_INFRAS_Transport_Mitigation_Options_Kenya.pdf

# create a dictionary for GIZ data - pkm derived from vkm and occupancy rates

# BILLION pkm
kenya_pkm = {'Walk': 22,  # guess - no data. Assume roughly twice that of UK person.
       'Bike': 2,  # guess - no data. Assume roughly that of UK person.
       'CarDriver': 0.65 * 9.903,  # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'CarPass': 0.3 * 9.903,  # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'Moto': 7.542,   # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'LocalBus': 10.357 + 18.609,   # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'Coach': 6.075,  # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'RailTube': 0.113,  # See Excel sheet data/sources/2021_GIZ_TrIGGER_Kenya-withworkings.xlsx
       'OtherPriv': 0.02 * 9.903, 
       'Taxi': 0.03 * 9.903, 
       'Air': 0.05 * 9.842,  # UNDESA - assume 5% of total air pkm is domestic
       'OtherPub': 0.001}

In [25]:
# now we will sum up the td's in the uk data, to derive multiplications from the UK data to the Kenya data
Kenya_pop = 43700000  # 2012
UK_pop = 63700000  # 2012
km_per_mile = 1.60934 # the td data is in miles...

for mode in mode_scaling:
    
    uk_td_by_mode = mode_shares[(mode_shares.Year == 2012) & (mode_shares.modeID == mode)]['td'].sum() * km_per_mile  # this is in MILES so convert to km!!!!!
    
    # fill data in mode_scaling dictionary
    mode_scaling[mode] = (kenya_pkm[modeID_lookup[mode]] * 10**9 / Kenya_pop) / uk_td_by_mode

In [26]:
mode_scaling

{1: 1.9497685259717847,
 2: 0.5373066085729336,
 3: 0.027763272622613577,
 4: 0.023170865017151713,
 5: 2.780564418548479,
 6: 1.413125463726313,
 7: 1.3196025331477448,
 8: 0.0025470928700848947,
 9: 0.02962646401405987,
 10: 0.08720364970251844,
 11: 0.24045471297550713,
 12: 0.00018894024085735914}

In [27]:
for mode in mode_scaling:
    mode_shares.loc[mode_shares[mode_shares.modeID == mode].index.tolist(), 'td'] *= mode_scaling[mode]

mode_shares

,ScenarioID,CountryID,PolicyID,Year,triplenID,modeID,td
0,1,9,0,2012,1,1,109.619804
1,1,9,0,2012,1,2,0.684038
2,1,9,0,2012,1,3,0.344905
3,1,9,0,2012,1,4,0.172820
4,1,9,0,2012,1,5,0.106883
...,...,...,...,...,...,...,...
8539,1,9,0,2100,8,8,0.564366
8540,1,9,0,2100,8,9,1.787392
8541,1,9,0,2100,8,10,0.328050
8542,1,9,0,2100,8,11,3.387087


In [28]:
mode_shares.to_csv(f'./../../TEAM-Kenya_data_out/base-case-kenya-data/mode_shares_triplength_team_kenya.csv', index=False)